In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/users/kp_epcc/aq_stripes/visualisation_code/')
    
from plotting_routines_cities import (plot_absolute_bar_plot,
                                      plot_aq_stripes_withline,
                                      plot_aq_stripes_withline_withindicativecolourbar,
                                      plot_summary_statistics)


import settings

from settings import OUTPUT_DATA_DIR


In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
import os
import pandas as pd 
import textwrap


def create_custom_cmap():

    colors = [
        '#a4ffff',  # 1 Blue
        '#b0dae9',  # 2 Blue
        '#b0ceed',  # 3 Blue
        '#F9E047',  # 4 Yellow
        '#f2c84b',  # 5 Yellow
        '#f1a63f',  # 6 Orange      
        '#E98725',  # 7 Orange
        '#af4553',  # 8 (Red)
        '#863b47',  # 9 (Dark Red)
        '#673a3d',  # 10 (Darker Red)
        '#462f30',  # 11 (Very Dark Brown)
        '#252424',  # 12 (Almost Black)
    ]

    num_colors = len(colors)
    custom_cmap = ListedColormap(colors)    
    return custom_cmap



In [ ]:
from collections import Counter

# Define the path to cities data file
cities_file_path = os.path.join('List_of_Cities_Continent.txt')

# Create an empty dictionary to store city data
cities_continent = {}

# Load the cities dictionary
with open(cities_file_path, 'r') as file:
    file_content = file.read()
    exec(file_content)  # Executes the cities_continent dictionary assignment

# Count the number of unique entries in the dictionary
unique_entries_count = len(cities_continent)

print(f"Number of unique entries in cities_continent: {unique_entries_count}")

# Check for duplicate city entries
city_names = list(cities_continent.keys())
city_counts = Counter(city_names)
duplicates = {city: count for city, count in city_counts.items() if count > 1}
if duplicates:
    print("Duplicate entries found:")
    for city, count in duplicates.items():
        print(f"{city} appears {count} times")
else:
    print("No duplicate entries found.")

In [ ]:
%reload_ext autoreload

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Read in CMIP (model) and VAND (satellitte) data
dir_path = "/home/users/kp_epcc/aq_stripes/input_data/"

ds_VAND = xr.open_dataset(dir_path + "vand_annual_mean_output.nc")
ds_CMIP = xr.open_dataset(dir_path + "cmip_annual_mean_output.nc")

# Set the time coordinates to just the year value
ds_CMIP['time'] = ds_CMIP['time'].dt.year
ds_VAND['time'] = ds_VAND['time'].dt.year

# Make a long time axis from start of CMIP to end of VAND
years = np.arange(ds_CMIP['time'].min(), ds_VAND['time'].max() + 1)

# Define the time period used to average for the weighting
time_period = slice(2000, 2002)  
time_period_list = [2000, 2001, 2002]  # Only used for plotting

#  Call colour scheme
custom_cmap = create_custom_cmap()

# Dictionary to hold combined data for each city
city_combined_data = {}

# Create a DataFrame to store the city and corresponding data_ratio
df_data_ratio = pd.DataFrame(columns=['City', 'Data_Ratio'])
df_format1 = pd.DataFrame({'Year': years})

count=0
for city, info in cities_continent.items():

    count += 1
    print(f"Processing {city}", f"Total cities processed: {count}")    
    
    latitude = info['lat']
    longitude = info['lon']
    continent = info['continent'].replace(' ', '_')

    print("")
    print("******************************************************")
    print("latitude",latitude)
    print("longitude", longitude)
    print("City = ",city)
    print("Continent = ",continent)
    print("")   
    
    # Extract the value at the correct lat / lon of interest
    
    # Use linear interpolation for CMIP model data and nearest for VAND satellitte data
    data_CMIP = ds_CMIP['PM25_CMIP'].interp(latitude=latitude, longitude=longitude, method='linear')    
    data_VAND = ds_VAND['PM25_VAND'].sel(latitude=latitude, longitude=longitude, method='nearest')
    
    # Calculate the three year average value (time_period) for the lat / lon of interest
    data_CMIP_mean = data_CMIP.sel(time=time_period).mean(dim='time')
    data_VAND_mean = data_VAND.sel(time=time_period).mean(dim='time')   
   
    # Calculate the ratio of the averaged CMIP data to the averaged VAND data for the lat / lon of interest 
    data_ratio = data_VAND_mean / data_CMIP_mean

    if data_ratio < 2:
        uncertainty_classification = "Low Uncertainty"
    elif data_ratio < 4:
        uncertainty_classification = "Moderate Uncertainty"
    else:
        uncertainty_classification = "High Uncertainty"

    print(uncertainty_classification)

    
    print("Calculated data ratio (VAND/CMIP):")
    print(data_ratio)  

    # Weight the CMIP data by the rato of VAND / CMIP
    CMIP_data_weighted = data_CMIP * data_ratio

    print("data_VAND_mean",data_VAND_mean, "data_CMIP_mean",data_CMIP_mean," data_ratio = ",data_ratio)
    print("CMIP_data_weighted size and dimensions before concatenation:", CMIP_data_weighted.size, CMIP_data_weighted.dims)
    print("CMIP_data_weighted time range:", CMIP_data_weighted.time.min().values, "to", CMIP_data_weighted.time.max().values)
    print("data_VAND size and dimensions before concatenation:", data_VAND.size, data_VAND.dims)
    print("data_VAND time range:", data_VAND.time.min().values, "to", data_VAND.time.max().values)

    # Concatenate data_weighted and data_VAND at the year 2000
    data_combined = xr.concat([CMIP_data_weighted.sel(time=slice(None, 2000)), data_VAND.sel(time=slice(2001, None))], dim='time')
    
    # Print the size and dimensions of the array after concatenation
    print("data_combined size and dimensions after concatenation:", data_combined.size, data_combined.dims)
    print("data_combined time range:", data_combined.time.min().values, "to", data_combined.time.max().values)
   
    # Store the combined data in the dictionary
    city_combined_data[city] = data_combined

    # Calculate the maximum value of your data
    max_value = max(data_combined.max(), data_CMIP.max(), data_VAND.max())

    # Code to create plots to check VAND, CMIMP and combined values   
    plt.ylim(0, max_value * 1.2)  
    plt.plot(years, data_combined, label=f"{city} Combined", color="darkgrey", linewidth=4)
    plt.plot(data_CMIP['time'], data_CMIP, label=f"{city} Model", linewidth=1.5, linestyle='-.', color="blue")
    plt.plot(data_VAND['time'], data_VAND, label=f"{city} Satellite", linewidth=1.5, linestyle='--', color="red")
    plt.scatter(time_period_list, [data_CMIP_mean] * len(time_period_list), color='darkblue', label=f"{city} Model 3-year Mean", zorder=5)
    plt.scatter(time_period_list, [data_VAND_mean] * len(time_period_list), color='darkred', label=f"{city} Satellite 3-year Mean", zorder=5)
    extra_info = f"{uncertainty_classification}, Data Ratio = {data_ratio:.1f}"
    plt.plot([], [], label=extra_info, color='none')  # Invisible plot for the extra legend entry

    plt.xlabel('Year')
    plt.ylabel('Annual mean PM2.5 (ug m-3)')
    plt.title('PM2.5 Timeseries')
    plt.legend(fontsize=8)
    plt.grid(True)
    plot_title = city    
    plt.savefig(OUTPUT_DATA_DIR + plot_title + '_'+continent+ '_CMIP_VAND_May.png', dpi=300, bbox_inches='tight', pad_inches=0.05)
    plt.show()   
    plt.close()
    
    # Stripes plotting routines
    plot_absolute_bar_plot(data_combined.values, data_combined['time'].values, custom_cmap, plot_title, continent)   
    plot_aq_stripes_withline(data_combined.values, data_combined['time'].values, custom_cmap, plot_title, continent)
    plot_aq_stripes_withline_withindicativecolourbar(data_combined.values, data_combined['time'].values, custom_cmap, plot_title, continent, uncertainty_classification, data_ratio)   
    plot_summary_statistics(data_combined.values, data_combined['time'].values, custom_cmap, plot_title, continent)

    # Append the city and its data_ratio to the DataFrame
    df_data_ratio = pd.concat([df_data_ratio, pd.DataFrame({'City': [city], 'Data_Ratio': [data_ratio.values]})], ignore_index=True)

    # Add the city's data to the DataFrame in wide format
    df_format1[city] = data_combined.sel(time=years).values

# Write cities PM2.5 data and sat to model data ratio to file
df_data_ratio.to_excel(OUTPUT_DATA_DIR + "Cities_Data_Ratios.xlsx", index=False)
df_format1.to_excel(OUTPUT_DATA_DIR + "Cities_Wide_Format_Data.xlsx", index=False)  

# Count the number of files containing the word "CMIP"
cmip_file_count = sum(1 for file_name in os.listdir(OUTPUT_DATA_DIR) if "CMIP" in file_name)

# Compare and print results
if cmip_file_count == unique_entries_count:
    print(f"The number of files with 'CMIP' ({cmip_file_count}) matches the unique city count ({unique_entries_count}).")
else:
    print(f"Mismatch: {cmip_file_count} files with 'CMIP', but {unique_entries_count} unique city entries.")
